In [3]:
import pyvisa
from nanodrivers.visa_drivers.anapico import ANAPICO
from nanodrivers.visa_drivers.dc import DC
from nanodrivers.visa_drivers.vna import VNA
from nanodrivers.visa_drivers.signal_analyser import Anri
# from nanodrivers.visa_drivers.signal_analizar import Anri

#math
import numpy as np
from numpy import *
from scipy.optimize import curve_fit

#plotting
import matplotlib.pyplot as plt
import matplotlib

#matlab import
import scipy.io
from scipy.io import savemat
from scipy.optimize import minimize

#file_store
import os

#time
import time
from datetime import datetime, timedelta

from ctypes import *
%matplotlib inline

In [5]:
vna = VNA()
vna.idn()
# vna.dump()

Connection exist: Rohde-Schwarz,ZNB20-2Port,1311601062102129,3.45



In [6]:
anapico = ANAPICO()
anapico.idn()
anapico.dump()['channel_status']

Connection exist: AnaPico AG,APMS20G,3C6-0B4F40003-0985,0.4.204



array([0., 0., 0., 0.])

In [8]:
dc = DC()
dc.idn()

Connection exist: HEWLETT-PACKARD,33120A,0,7.0-5.0-1.0



# 1st tone

In [9]:
vna.set_lin()

folder = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise"
exp_name = '2nd_tone'
path = folder+'\\'+exp_name
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\2nd_tone created!


In [26]:
volts = np.linspace(0, 4, 80)
pump_freq = np.linspace(11e9, 12e9, 100)
vna.set_band(100)
vna.set_nop(200)

freqs = vna.set_cw_freq(6e9)

state = dict()
state['pump_freq'] = pump_freq
state['CW_freq'] = vna.get_cent_freq

state['volts'] = volts
state['vna'] = vna.dump
state['DC_dev'] = dc.dump
state['anapico'] = anapico.dump()
state['temp'] = 8


mag_1t = np.empty((len(volts),len(pump_freq)))
pha_1t = np.empty((len(volts),len(pump_freq)))

state['mag'] = mag_1t
state['mag'] = pha_1t

start_time = datetime.now()
file = r'2nd_tone_no_AC'+ start_time.strftime(r'%Y-%m-%d-%H-%M-%S') + '.mat'
file_name = path +'\\' + file

start_time = datetime.now()  


anapico.set_freq(1, pump_freq[0])
anapico.set_power(1, +10)
anapico.set_on(1)

for i in range(len(volts)):
    dc.set_volt(volts[i])
    curr_time = datetime.now()
    print('{} -> new volt: {}'.format(curr_time.strftime(r'%H-%M-%S'),  volts[i]))
    time.sleep(10)
    for j in range(len(pump_freq)):
        
        anapico.set_freq(1, pump_freq[j])
        mag, pha = vna.get_data()
        mag_1t[i, j], pha_1t[i, j] = np.mean(mag), np.mean(pha)

        state['mag'] = mag_1t
        state['mag'] = pha_1t

        savemat(file_name, state)
    
    
dc.set_volt(0)   
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

21-31-11 -> new volt: 0.0
21-35-16 -> new volt: 0.05063291139240506
21-39-21 -> new volt: 0.10126582278481013
21-43-27 -> new volt: 0.1518987341772152
21-47-32 -> new volt: 0.20253164556962025
21-51-37 -> new volt: 0.25316455696202533
21-55-43 -> new volt: 0.3037974683544304
21-59-48 -> new volt: 0.35443037974683544
22-03-55 -> new volt: 0.4050632911392405
22-08-00 -> new volt: 0.45569620253164556
22-12-06 -> new volt: 0.5063291139240507
22-16-11 -> new volt: 0.5569620253164557
22-20-16 -> new volt: 0.6075949367088608
22-24-21 -> new volt: 0.6582278481012658
22-28-26 -> new volt: 0.7088607594936709
22-32-32 -> new volt: 0.7594936708860759
22-36-37 -> new volt: 0.810126582278481
22-40-42 -> new volt: 0.8607594936708861
22-44-47 -> new volt: 0.9113924050632911
22-48-55 -> new volt: 0.9620253164556962
22-53-00 -> new volt: 1.0126582278481013
22-57-05 -> new volt: 1.0632911392405062
23-01-10 -> new volt: 1.1139240506329113
23-05-16 -> new volt: 1.1645569620253164
23-09-21 -> new volt: 1.21

In [29]:
%matplotlib qt
font = {'size'   : 15}
matplotlib.rc('font', **font)
fig, ax = plt.subplots(1, 2, figsize=(25,13))
c = ax[0].pcolor(pump_freq/1e9, volts,   mag_1t, cmap='viridis')
ax[0].set_ylabel(r'DC, V')
ax[0].set_xlabel(r'Signal frequency, GHz')

c1 = ax[1].pcolor(pump_freq/1e9, volts,   pha_1t, cmap='viridis')
ax[1].set_ylabel(r'DC, V')
ax[1].set_xlabel(r'Signal frequency, GHz')

fig.colorbar(c, ax=ax[0])
fig.colorbar(c1, ax=ax[1])

ax[0].set_title(r'$|S_{21}|$')
ax[1].set_title(r'$\angle S_{21}|$')

Text(0.5, 1.0, '$\\angle S_{21}|$')

In [18]:
vna.set_lin()

In [24]:
vna.set_stop_freq(5e9)
vna.set_band(10)
vna.set_nop(2000)
mag, pha = vna.get_data()
fre = vna.get_freq()

In [25]:
plt.plot(fre, mag)